In [ ]:
!git clone https://huggingface.co/decapoda-research/llama-7b-hf

In [ ]:
!pip install datasets loralib sentencepiece
!pip install transformers
!pip install peft
!pip install bitsandbytes

In [ ]:
from peft import PeftModel
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
import textwrap

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LlamaForCausalLM.from_pretrained(
    "decapoda-research/llama-7b-hf",
    load_in_8bit=True,
    device_map="auto",
)
#model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")

In [ ]:

def generate_text(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
    for s in generation_output.sequences:
        print(tokenizer.decode(s))

In [ ]:
input_text = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
could you write some python code to print the sequence of fibonacci ?

### Response:"""
generate_text(input_text)